# Example of how to use the ALoNe framework
In this notebook, we demonstrate how to use the Analysis of Local Neighbourhoods (ALoNe) framework. For that we first generate a synthetic dataset (a set of random points in 3D), then use voro++ to generate organisational features, which we then explore a little bit.

In [1]:
import ALoNe
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import vedo
vedo.settings.notebookBackend = 'k3d'
vedo.settings.k3dPointShader = '3d'

## Generate random points

In [4]:
x = 20  # number of points
points = np.array(list(np.ndindex(x,x,x))) * 10


In [12]:
vedo.show(vedo.Points(points).color('r'), vedo.Points(np.array(list(np.ndindex(x//2,x//2,x//2))) * 10 + 5).color('b'))

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

3D rendering of the initial point distribution. All points are used in the following processing. Colours are just to illustrate the difference in point density in one corner.

In [14]:
points = np.vstack([points, np.array(list(np.ndindex(x//2,x//2,x//2))) * 10 + 5])

In [15]:
noise_factor = 5
points_noise = points + (np.random.random(points.shape) * noise_factor)

In [16]:
vedo.show(vedo.Points(points_noise).color('b'))

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

3D rendering of the noisy point distribution.

In [18]:
df_reg = pd.DataFrame(points, columns=list('xyz'))
df_reg['cell id'] = df_reg.index + 1
df_reg['sample'] = 'regular'

df_noise = pd.DataFrame(points_noise, columns=list('xyz'))
df_noise['cell id'] = df_noise.index + 1
df_noise['sample'] = 'noisy'

df = pd.concat([df_reg, df_noise])

In [20]:
df  # This is the main data frame that we will be working with.

,x,y,z,cell id,sample
0,0.000000,0.000000,0.000000,1,regular
1,0.000000,0.000000,10.000000,2,regular
2,0.000000,0.000000,20.000000,3,regular
3,0.000000,0.000000,30.000000,4,regular
4,0.000000,0.000000,40.000000,5,regular
...,...,...,...,...,...
9995,96.617447,96.444925,59.777410,9996,noisy
9996,98.303484,97.045749,67.891763,9997,noisy
9997,98.295800,97.934389,78.911608,9998,noisy
9998,97.831606,95.462480,86.726837,9999,noisy


`df` columns are 
* `x, y, z`: coordinates
* `cell id`: ID of individual point. Has to be > 0 and integer for voro++
* `sample`: disciminate different samples by category

## Generate restricted Voronoi diagram and organisational features
First, we generate a restricted Voronoi diagram using our modified version of voro++. The modification is that we can call a radially restricted Voronoi diagram from the command line (this is not implemented in the original voro++ code).
Then we evaluate the Voronoi diagram and generate features from it. Moreover, we generate a kernel density estimation at different length scales (multi scale density). 
We do this for every sample individually.

In [21]:
list_df = []
for sid in df['sample'].unique():
    sdf = df.loc[df['sample'] == sid].copy()
    sdf = ALoNe.Voronoi.voronoi_restricted(sdf)
    sdf = ALoNe.FeatureGeneration.voronoi_features(sdf)
    ALoNe.FeatureGeneration.multi_scale_density(sdf, np.arange(5,44,5))
    list_df.append(sdf)
df = pd.concat(list_df)

neighbourhood centroid offset: 100%|██████████| 10000/10000 [00:01<00:00, 6529.14it/s]


## Visualisation
Here we illustrate the distributions of organisational features on our test data in 3D by colouring the points according to their feature values.

In [23]:
df.columns

Index(['x', 'y', 'z', 'cell id', 'sample', 'shell 5', 'shell 10', 'shell 15',
       'shell 20', 'shell 25', 'shell 30', 'shell 35', 'shell 40',
       'vertex number', 'edge number', 'edge distance', 'face number',
       'voronoi surface area', 'voronoi volume', 'voronoi sphericity',
       'x centroid', 'y centroid', 'z centroid', 'centroid offset',
       'neigbour cell ids', 'neighbour boundaries', 'coordinates vertices',
       'vertices per face', 'point type', 'n neighbours',
       'density voronoi mean', 'density voronoi std',
       'neighbourhood voronoi volume mean', 'neighbourhood voronoi volume std',
       'neighbourhood voronoi sphericity mean',
       'neighbourhood voronoi sphericity std',
       'neighbourhood n neighbours mean', 'neighbourhood n neighbours std',
       'neighbourhood centroid offset mean',
       'neighbourhood centroid offset std'],
      dtype='object')

In [36]:
vedo.show(ALoNe.Visualisation.show_features(
    df.loc[df['sample'] == 'noisy'],
    ['voronoi volume']),
)

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…